# 월별 물동량 추이
### 크롤링

In [107]:
import numpy as np
import pandas as pd
from selenium import webdriver #Selenium의 웹 드라이버를 사용하기 위한 모듈을 임포트
from selenium.webdriver.common.by import By #Selenium에서 사용하는 By 클래스를 임포트합. 웹 요소를 검색하는데 사용.
from selenium.webdriver.common.keys import Keys #키보드 입력 제어를 위해 Keys 클래스 임포트
from selenium.webdriver.chrome.service import Service #Chrome 드라이버 서비스를 사용하기 위한 모듈 임포트
from selenium.webdriver.chrome.options import Options #Chrome 드라이버 옵션을 설정하기 위한 클래스 임포트
from webdriver_manager.chrome import ChromeDriverManager #Chrome 드라이버를 자동으로 설치 및 관리하는데 사용되는 드라이버 매니저 임포트
from bs4 import BeautifulSoup #Beautiful soup 크롤링을위해
import time

In [126]:
def get_monthly_shipments():
    # 브라우저 꺼짐 방지
    chrome_options = Options()
    chrome_options.add_experimental_option("detach", True)
    # 불필요한 에러 메시지 없애기
    chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(options=chrome_options)
    driver.get('https://www.chainportal.co.kr/portstat/nexacro/index.html?screenid=screen_stat')
    driver.maximize_window()

    driver.implicitly_wait(20)
    # 로그인 진행
    id_btn=driver.find_element(By.CSS_SELECTOR,'#mainframe\.frameIndex\.form\.divCenter\.form\.divLogin\.form\.edId\:input')
    pw_btn=driver.find_element(By.CSS_SELECTOR,'#mainframe\.frameIndex\.form\.divCenter\.form\.divLogin\.form\.edPw\:input')
    id_btn.send_keys('zoqxls98')
    pw_btn.click()
    time.sleep(1)
    pw_btn.send_keys('Rlatjddlf527&')
    login_btn=driver.find_element(By.CSS_SELECTOR,'#mainframe\.frameIndex\.form\.divCenter\.form\.divLogin\.form\.btnLogin\:icontext')
    login_btn.click()

    time.sleep(10)
    btn_1=driver.find_element(By.CSS_SELECTOR,'#mainframe\.frameIndex\.form\.divFrameLeft\.form\.divLeft\.form\.grdTree\.body\.gridrow_2\.cell_2_1\:text')
    btn_1.click()
    time.sleep(2)
    btn_2=driver.find_element(By.CSS_SELECTOR,'#mainframe\.frameIndex\.form\.divFrameLeft\.form\.divLeft\.form\.grdTree\.body\.gridrow_4\.cell_4_1\:text')
    btn_2.click()

    time.sleep(5)
    text_btn=driver.find_elements(By.CLASS_NAME,'nexatextitem')
    time.sleep(10)
    divide_btn=text_btn[14]
    divide_btn.click()
    
    soup_list=[]
    year_list=list(range(2018,2024))
    for i in year_list:
        time.sleep(5)
        year_btn=driver.find_elements(By.CLASS_NAME,'nexainput')[6]
        year_btn.click()
        year_btn.clear()
        time.sleep(5)
        year_btn.send_keys(str(i))
        time.sleep(5)
        search_btn=text_btn[13]
        search_btn.click()
        time.sleep(3)
        df=pd.DataFrame()
        for i in range(12):
            rows=driver.find_element(By.ID,f'mainframe.frameIndex.form.divMain.form.win80020200_0_185.form.divWork.form.grdMain.body.gridrow_{i}').text
        time.sleep(5)
        soup_list.append(soup)
        print(str(i)+'done')
    driver.close()    
    return soup_list

In [127]:
soups=get_monthly_shipments()

2018done
2019done
2020done
2021done
2022done
2023done


In [145]:
soups[0]

<html lang="ko"><!-- LANGUAGE : locale setting (value of location property in Environment) --><head>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="user-scalable=0,initial-scale=1, target-densitydpi=device-dpi,minimum-scale=1" name="viewport"/>
<!-- META_CHECKVERSION : Add if value of checkversion property in Envrionment is true -->
<meta content="no-cache" http-equiv="pragma"/>
<!-- FAVICON_ICON : favion setting (value of icon property in Environment. add to value of icon property in Environment) -->
<!-- USERFONT_STYLE : userfont style setting (value of userfontid property in Environment. crate contents contained in xfont file specified userfontid property in Environment) -->
<style id="userfont" type="text/css">
        @font-face {
          font-family : 'NanumMoogoonghwa';
          font-style : normal;
          font-weight : 700;
          src : local("NanumMoogoonghwa");
          src :

In [161]:
a = soups[0].select('div#mainframe.frameIndex.form.divMain.form.win80020200_0_264.form.divWork.form.grdMain.body > div.nexacontainer > div.nexainnercontainer> div#mainframe.frameIndex.form.divMain.form.win80020200_0_264.form.divWork.form.grdMain.body.gridrow_0')

for i in a:
    print(i.text)

In [154]:
soups[1].select('div.mainframe.frameIndex.form.divMain.form.win80020200_0_185.form.divWork.form.grdMain.body.gridrow_0.cell_0_0:text')


NotImplementedError: ':text' pseudo-class is not implemented at this time

In [122]:
months=[]
for i in texts:
    splited=i.split('\n')
    for j in range(len(splited)):
        if j%11==2:
            months.append(splited[j])

['구분\n합계\n1월\n2월\n3월\n4월\n5월\n6월\n7월\n8월\n9월\n10월\n11월\n12월\n수입\n5,329,475\n401,408\n410,606\n496,830\n463,123\n470,817\n441,138\n423,073\n440,981\n455,157\n428,694\n447,184\n450,464\n수출\n5,413,501\n419,471\n433,076\n471,957\n468,372\n475,547\n453,685\n449,243\n427,204\n462,628\n432,831\n473,899\n445,587\n수입환적\n6,205,377\n525,101\n442,852\n557,208\n537,680\n515,304\n504,451\n521,964\n509,580\n522,083\n538,320\n520,689\n510,144\n수출환적\n6,202,974\n493,326\n472,133\n551,532\n555,305\n528,268\n502,891\n502,122\n515,536\n525,638\n496,123\n551,887\n508,213\n소계\n23,151,327\n1,839,306\n1,758,667\n2,077,527\n2,024,480\n1,989,936\n1,902,165\n1,896,402\n1,893,301\n1,965,506\n1,895,968\n1,993,659\n1,914,408\n수입\n779,980\n428,356\n351,624\n수출\n822,556\n457,956\n364,600\n수입환적\n1,049,419\n558,645\n490,774\n수출환적\n1,017,969\n547,360\n470,609\n소계\n3,669,924\n1,992,317\n1,677,607\n수입\n-85.36\n6.71\n-14.36\n수출\n-84.80\n9.17\n-15.81\n수입환적\n-83.08\n6.38\n10.82\n수출환적\n-83.58\n10.95\n-0.32\n소계\n-84.14\n8.31\n-